# Read 

In [56]:
import pandas as pd
train = pd.read_csv("snli_1.0/snli_1.0_train.txt", sep='\t')

In [374]:
test = pd.read_csv("snli_1.0/snli_1.0_test.txt", sep='\t')

In [58]:
columns = ['gold_label','sentence1_binary_parse','sentence2_binary_parse','sentence1_parse','sentence2_parse','sentence1','sentence2','captionID','pairID','label1','label2','label3','label4','label5']

In [59]:
train.columns=columns
test.columns=columns

In [27]:
train['label1'].value_counts()

contradiction    183384
entailment       183384
neutral          183384
Name: label1, dtype: int64

In [28]:
test['label1'].value_counts()

neutral          3334
entailment       3333
contradiction    3333
Name: label1, dtype: int64

In [30]:
train.shape

(550152, 14)

In [55]:
train[6:9]

,gold_label,sentence1_binary_parse,sentence2_binary_parse,sentence1_parse,sentence2_parse,sentence1,sentence2,captionID,pairID,label1,label2,label3,label4,label5
6,contradiction,( ( A boy ) ( ( is ( ( jumping ( on skateboard...,( ( The boy ) ( ( ( skates down ) ( the sidewa...,(ROOT (S (NP (DT A) (NN boy)) (VP (VBZ is) (VP...,(ROOT (S (NP (DT The) (NN boy)) (VP (VBZ skate...,A boy is jumping on skateboard in the middle o...,The boy skates down the sidewalk.,3691670743.jpg#0,3691670743.jpg#0r1c,contradiction,NaN,NaN,NaN,NaN
7,entailment,( ( A boy ) ( ( is ( ( jumping ( on skateboard...,( ( The boy ) ( ( does ( a ( skateboarding tri...,(ROOT (S (NP (DT A) (NN boy)) (VP (VBZ is) (VP...,(ROOT (S (NP (DT The) (NN boy)) (VP (VBZ does)...,A boy is jumping on skateboard in the middle o...,The boy does a skateboarding trick.,3691670743.jpg#0,3691670743.jpg#0r1e,entailment,NaN,NaN,NaN,NaN
8,neutral,( ( A boy ) ( ( is ( ( jumping ( on skateboard...,( ( The boy ) ( ( is ( wearing ( safety equipm...,(ROOT (S (NP (DT A) (NN boy)) (VP (VBZ is) (VP...,(ROOT (S (NP (DT The) (NN boy)) (VP (VBZ is) (...,A boy is jumping on skateboard in the middle o...,The boy is wearing safety equipment.,3691670743.jpg#0,3691670743.jpg#0r1n,neutral,NaN,NaN,NaN,NaN


In [51]:
train['sentence1'].values[6]

'A boy is jumping on skateboard in the middle of a red bridge.'

In [52]:
train['sentence2'].values[6]

'The boy skates down the sidewalk.'

In [53]:
train['sentence2'].values[7]

'The boy does a skateboarding trick.'

In [54]:
train['sentence2'].values[8]

'The boy is wearing safety equipment.'

In [ ]:
ідеї


1вчити абстакції наприклад  
    broken down airplane = person is outdoors
    on a horse =  jumps on a horse
    
2 дивитися по ворднету наскільки далеко ці слова

3 зробити реплейс усіх займеників одним словом і іменників теж оскількі в цих реченнях мова иде про одне и теж

4 Більшість речень має 2 стейтменти, можливо варто їх окремо розглядати і порівнювати окремо напрклад от 
    in the middle of a red bridge !=  down the sidewalk
    
    
    умови будуть такі 
            якщо в 1му і другому речені є ця частина речення тоді порівнюємо
            якщо нема вважаємо що ця частина збігається
            
    це можна також використати як окрему фічу
    
5 не використовувати голден трі - оскільки потім я зможу цю модель перевикористати


# Sbj - verb - object spliting

In [286]:
import spacy
from nltk import Tree
en_nlp = spacy.load('en_core_web_sm')

In [419]:
def get_roots(nlp_doc):
    root = None 
    for token in nlp_doc:
        if token.dep_.lower() =='root':
            root = token
            break
    conjs = [child for child in nlp_doc if child.dep_ in ['conj','advcl','acl','advmod','relcl']] 
    conjs = conjs+[child for child in nlp_doc if (child.dep_ in ['pobj']) and (child.pos_ in ['VERB'])] 
    conjs.insert(0,root)
    return conjs

In [334]:
def merge_nouns(doc):
    viewed_tokens_for_merge=[]
    merge_ranges=[]
    for token in doc:
        if token in viewed_tokens_for_merge:
            continue
        tmp_compaund_token=token
        
        last_token=tmp_compaund_token
        while((tmp_compaund_token.dep_ in ['compound'])):#'nummod'
            viewed_tokens_for_merge.append(token)
            if (tmp_compaund_token.head.pos_ in ['NOUN','PROPN']):
                last_token=tmp_compaund_token.head
            tmp_compaund_token = tmp_compaund_token.head
            
        if token!=tmp_compaund_token:
            merge_ranges.append((token.idx, last_token.idx+last_token.__len__()))
            
    for merge_range in merge_ranges:
        doc.merge(merge_range[0],merge_range[1])
    return doc

def merge_werbs(doc):
    merge_ranges=[]
    for token in doc:
        if(token.head.tag_ in ['VERB','VBG']) and (token.dep_ in ['prt']):
            start = token.idx
            end = token.head.idx
            if start<end:
                merge_ranges.append((token.idx, token.head.idx+token.head.__len__()))
            else:
                merge_ranges.append((token.head.idx, token.idx+token.__len__()))
    
    for merge_range in merge_ranges:
        doc.merge(merge_range[0],merge_range[1])
    return doc

def get_doc(text):
    nlp_doc = en_nlp(text)
    nlp_doc = merge_nouns(nlp_doc)
    nlp_doc = merge_werbs(nlp_doc)  
    return nlp_doc

In [327]:
dep_types={
    'subj_dep':['nsubj','nsubjpass','compound'],
    'vb_dep':['prt','pobj'],
    "obj_dep":['dobj','prep','acomp']
}

In [328]:
dep_types_important_pos={
    'subj_dep':['NOUN','ADJ','PROPN'],
    'vb_dep':['VERB'],
    "obj_dep":['NOUN','ADJ','PROPN']
}

In [413]:
text = 'An old man with a package poses in front of an advertisement.'
print(text)
nlp_doc = get_doc(text)
[(i, i.dep_, i.pos_,i.head) for i in nlp_doc]

An old man with a package poses in front of an advertisement.


[(An, 'det', 'DET', man),
 (old, 'amod', 'ADJ', man),
 (man, 'ROOT', 'NOUN', man),
 (with, 'prep', 'ADP', man),
 (a, 'det', 'DET', poses),
 (package, 'compound', 'NOUN', poses),
 (poses, 'pobj', 'VERB', with),
 (in, 'prep', 'ADP', poses),
 (front, 'pobj', 'NOUN', in),
 (of, 'prep', 'ADP', front),
 (an, 'det', 'DET', advertisement),
 (advertisement, 'pobj', 'NOUN', of),
 (., 'punct', 'PUNCT', man)]

In [420]:
nlp_doc = get_doc(text)
roots = get_roots(nlp_doc)
roots

[man, poses]

In [381]:
def add_dep(token, parts,dep_type):
    parts[dep_type].append(token)
    for sub_tokens in token.children:
        if sub_tokens.pos_ not in ['VERB']:
            add_dep(sub_tokens,parts,dep_type)
        else:
            parts[dep_type].append(sub_tokens)
        

In [440]:
def can_replace_subject(first,second):
    l =  second.idx - first.idx - len(first.text)
    if (l < 10):
        return True
    else:
        return False

In [435]:
def get_parts(nlp_doc,roots):
    all_parts=[]
    for root in roots:
        parts={}

        for dep_type in dep_types:
            parts[dep_type]=[]
            for token in root.children:
                if  token.dep_ in dep_types[dep_type]:
                    add_dep(token, parts,dep_type)
        
        # if root was set as an Noun
        if root.tag_ != 'NN':
            parts['vb_dep'].insert(0,root)
        else:
            parts['subj_dep'].insert(0,root)
        
        #remove irrelevant tags  
        all_parts.append(cleanup_parts(parts))
        
    # duplicate noun from first item if missed
    for i in range(1,len(all_parts)):
        
        if len(all_parts[i]['subj_dep'])==0:
            all_parts[i]['subj_dep']=all_parts[i-1]['subj_dep']
        elif len(all_parts[i-1]['subj_dep'])>0 and can_replace_subject(all_parts[i-1]['subj_dep'][0],all_parts[i]['subj_dep'][0]):
            # should optimized better 
            all_parts[i]['subj_dep']=all_parts[i-1]['subj_dep']
        
    #cleanup parts only with noun due to incorect work of pos tagger
    result=[]
    
    for parts in all_parts:
        if (((len(parts['vb_dep'])>0) and  (parts['vb_dep'][0].text!='is')) or (len(parts['obj_dep'])>0)):
            result.append(parts)
        
        
    return result

In [173]:
def cleanup_parts(parts):
    clean_part={}
    for part_type in parts:
        clean_part[part_type]=[]
        for token in parts[part_type]:
            if token.pos_ in dep_types_important_pos[part_type]:
                clean_part[part_type].append(token)
    return clean_part

# get most common tags in each group(subj, vb, object)

In [194]:
most_frequent_tags = {
    'subj_dep':[],
    'vb_dep':[],
    "obj_dep":[]
}

for text in test['sentence2'].values:
    nlp_doc = get_doc(text)
    roots = get_roots(nlp_doc)
    for root in roots:
        parts = get_parts(nlp_doc,root)
        for part_type in parts:
            most_frequent_tags[part_type]=most_frequent_tags[part_type]+[i.pos_ for i in parts[part_type]]
    

In [195]:
from collections import Counter
for part_type in most_frequent_tags:
    cnt = Counter(most_frequent_tags[part_type])
    print(part_type)
    print(cnt.most_common(10))
    print("====================")

subj_dep
[('NOUN', 9789), ('DET', 6925), ('ADJ', 1246), ('NUM', 973), ('ADP', 735), ('CCONJ', 288), ('VERB', 240), ('PRON', 204), ('PROPN', 116), ('ADV', 46)]
vb_dep
[('VERB', 8936), ('NOUN', 1263), ('PART', 258), ('ADP', 27), ('ADJ', 23), ('DET', 14), ('ADV', 13), ('PROPN', 7), ('INTJ', 3), ('CCONJ', 2)]
obj_dep
[('NOUN', 9537), ('ADP', 5582), ('DET', 5452), ('ADJ', 2527), ('VERB', 625), ('PART', 229), ('PROPN', 219), ('ADV', 168), ('PRON', 149), ('CCONJ', 112)]


Use only most frequet and important tags, fix root in those cases where root is NOUN

# Validate results 

In [436]:
for text in test['sentence2'].values[10:20]:
    print(text)
    nlp_doc = get_doc(text)
    roots = get_roots(nlp_doc)
    parts =get_parts(nlp_doc,roots)
    print(parts)
    print('======================================')

There is a statue that not many people seem to be interested in.
[{'subj_dep': [people, many], 'vb_dep': [seem], 'obj_dep': []}]
Tons of people are gathered around the statue.
[{'subj_dep': [Tons, people], 'vb_dep': [gathered], 'obj_dep': [statue]}]
A Land Rover is splashing water as it crosses a river.
[{'subj_dep': [Land Rover], 'vb_dep': [splashing], 'obj_dep': [water]}, {'subj_dep': [Land Rover], 'vb_dep': [crosses], 'obj_dep': [river]}]
A vehicle is crossing a river.
[{'subj_dep': [vehicle], 'vb_dep': [crossing], 'obj_dep': [river]}]
A sedan is stuck in the middle of a river.
[{'subj_dep': [sedan], 'vb_dep': [stuck], 'obj_dep': [middle, river]}]
A man playing banjo on the floor.
[{'subj_dep': [man], 'vb_dep': [playing], 'obj_dep': [banjo, floor]}]
A man playing guitar on stage.
[{'subj_dep': [man], 'vb_dep': [playing], 'obj_dep': [guitar, stage]}]
A man is performing for cash.
[{'subj_dep': [man], 'vb_dep': [performing], 'obj_dep': [cash]}]
A doctor is looking at a book
[{'subj_de

# Errors: 

In [211]:
The woman has been shot.
{'subj_dep': [woman], 'vb_dep': [shot], 'obj_dep': []}
======================================
The statue is offensive and people are mad that it is on display.
{'subj_dep': [statue], 'vb_dep': [is], 'obj_dep': [offensive]}
======================================
There is a statue that not many people seem to be interested in.
{'subj_dep': [], 'vb_dep': [is], 'obj_dep': []}
======================================
A Land Rover is splashing water as it crosses a river.
{'subj_dep': [], 'vb_dep': [splashing], 'obj_dep': [water]}
======================================
A man playing banjo on the floor.
{'subj_dep': [], 'vb_dep': [], 'obj_dep': []}
======================================
The man is inside.
{'subj_dep': [man], 'vb_dep': [is], 'obj_dep': []}
======================================
Three firefighters putting out a fire inside of a subway station.
{'subj_dep': [], 'vb_dep': [], 'obj_dep': []}
======================================
Three firefighters coming up from a subway station.
{'subj_dep': [], 'vb_dep': [], 'obj_dep': []}
======================================
Three firefighters playing cards inside a fire station.
{'subj_dep': [], 'vb_dep': [], 'obj_dep': []}
======================================
The boy is playing on the swings after school.
{'subj_dep': [boy], 'vb_dep': [playing], 'obj_dep': [swings, school]}
======================================
An archeologist wearing a hat squats to examine the site for a dig
{'subj_dep': [], 'vb_dep': [], 'obj_dep': []}
======================================
A squatting woman wearing a hat touching the ground.
{'subj_dep': [], 'vb_dep': [], 'obj_dep': []}
======================================
A woman wearing a sun bonnet planting a garden.
{'subj_dep': [], 'vb_dep': [], 'obj_dep': []}
======================================
a guy near a building stands by two other men
{'subj_dep': [guy, building], 'vb_dep': [stands], 'obj_dep': [men, other]}
======================================
a busy man stands with bodyguards
{'subj_dep': [man, busy], 'vb_dep': [stands], 'obj_dep': [bodyguards]}
======================================

SyntaxError: invalid syntax (<ipython-input-211-7a2101482491>, line 1)

In [443]:
tests=[
    ('The statue is offensive and people are mad that it is on display',
     [
        {'subj_dep': ['statue'], 'vb_dep': ['is'], 'obj_dep': ['offensive']},
        {'subj_dep': ['people'], 'vb_dep': ['are'], 'obj_dep': ['mad']}
     ]),
    ('A Land Rover is splashing water as it crosses a river.',
     [
        {'subj_dep': ['Land Rover'], 'vb_dep': ['splashing'], 'obj_dep': ['water']},
        {'subj_dep': ['Land Rover'], 'vb_dep': ['crosses'], 'obj_dep': ['river']}
     ]),
    ('A woman wearing a sun bonnet planting a garden',
     [
        {'subj_dep': ['woman'], 'vb_dep': ['planting'], 'obj_dep': ['garden']},
        {'subj_dep': ['woman'], 'vb_dep': ['wearing'], 'obj_dep': ['sun bonnet']}
     ]),
    ('Three firefighters putting out a fire inside of a subway station',
     [
        {'subj_dep': ['firefighters'], 'vb_dep': ['putting out'], 'obj_dep': ['fire']},
        {'subj_dep': ['firefighters'], 'vb_dep': [], 'obj_dep': ['subway station']}
     ]),
    ('An archeologist wearing a hat squats to examine the site for a dig',
     [
        {'subj_dep': ['archeologist'], 'vb_dep': ['wearing'], 'obj_dep': ['hat squats']},
        {'subj_dep': ['archeologist'], 'vb_dep': ['examine'], 'obj_dep': ['site', 'dig']}
     ]),
    ('A man poses in front of an ad.',
     [
        {'subj_dep': ['man'], 'vb_dep': ['poses'], 'obj_dep': ['front', 'ad']}
     ]),
    ('An old man with a package poses in front of an advertisement.',
     [
        {'subj_dep': ['man'], 'vb_dep': ['poses'], 'obj_dep': ['front', 'advertisement']}
     ]),
    ('There is a statue that not many people seem to be interested in.',
     [
        {'subj_dep': ['not','people', 'many'], 'vb_dep': ['interested'], 'obj_dep': ['statue']}
     ])
]



def get_actual(text):
    nlp_doc = get_doc(text)
    result= []
    for item in get_parts(nlp_doc,get_roots(nlp_doc)):
        result.append(str(item).replace("'",''))
    return result

def get_test(features):
    result= []
    for item in features:
        result.append(str(item).replace("'",''))
    return result

for test_text, features in tests:
    actual = get_actual(test_text)
    test_=get_test(features)
    if(actual != test_):
        print(test_text)
        print(actual)
        print(test_)
        print('++++++++++')
    else:
        print("cool bro")
    

cool bro
cool bro
cool bro
cool bro
cool bro
cool bro
cool bro
There is a statue that not many people seem to be interested in.
['{subj_dep: [people, many], vb_dep: [seem], obj_dep: []}']
['{subj_dep: [not, people, many], vb_dep: [interested], obj_dep: [statue]}']
++++++++++


In [399]:
#мабуть потрібно перефразовувати,  поки не знаю як обробити
#можливо застосувати page rank

from rake_nltk import Rake
r = Rake()
r.extract_keywords_from_text('There is a statue that not many people seem to be interested in.')
r.get_ranked_phrases()

['many people seem', 'statue', 'interested']

# Build features

In [455]:
test_idx=2
print(test['sentence1'].values[test_idx*3])
print(get_actual(test['sentence1'].values[test_idx*3]))
print('')

print(test['label1'].values[test_idx*3])
print(test['sentence2'].values[test_idx*3])
print(get_actual(test['sentence2'].values[test_idx*3]))
print('')

print(test['label1'].values[test_idx*3+1])
print(test['sentence2'].values[test_idx*3+1])
print(get_actual(test['sentence2'].values[test_idx*3+1]))
print('')

print(test['label1'].values[test_idx*3+2])
print(test['sentence2'].values[test_idx*3+2])
print(get_actual(test['sentence2'].values[test_idx*3+2]))
print('')
print('=========================')

An old man with a package poses in front of an advertisement.
['{subj_dep: [man], vb_dep: [poses], obj_dep: [front, advertisement]}']

entailment
A man poses in front of an ad.
['{subj_dep: [man], vb_dep: [poses], obj_dep: [front, ad]}']

neutral
A man poses in front of an ad for beer.
['{subj_dep: [man], vb_dep: [poses], obj_dep: [front, ad, beer]}']

contradiction
A man walks by an ad.
['{subj_dep: [man], vb_dep: [walks], obj_dep: [ad]}']



In [452]:
test['sentence2'].values[6]

'A man poses in front of an ad.'

In [449]:
get_actual(test['sentence2'].values[6])

['{subj_dep: [man], vb_dep: [poses], obj_dep: [front, ad]}']

In [450]:
get_actual(test['sentence2'].values[7])

['{subj_dep: [man], vb_dep: [poses], obj_dep: [front, ad, beer]}']

In [451]:
get_actual(test['sentence2'].values[8])

['{subj_dep: [man], vb_dep: [walks], obj_dep: [ad]}']

# steps

In [ ]:
1 довжина речення
2 якщо значень в масиві > 1  вибирати найбільш коротчі кобінації
3 по кожній х груп знайти вектори слів і прирівняти до гіпотези
4 те саме тільки з ворднетом
5 кількість значень

# Get similarity by words word2vec and wordnet

In [457]:
import sys
import os
import numpy as np
import gensim


model = gensim.models.KeyedVectors.load_word2vec_format('../../Projects/GoogleNews-vectors-negative300.bin', binary=True)

In [458]:
model.wv.similarity("dog", "cat")

/Users/grycshuknazar/anaconda2/envs/env_36/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


0.7609457089782209

In [464]:
from nltk.corpus import wordnet as wn
heart_types[i].wup_similarity(heart_types[j])

In [490]:
from nltk.corpus import wordnet as wn

wordnet_tags={
    'ADJ':wn.ADJ,
    'ADV':wn.ADV,
    'NOUN':wn.NOUN,
    'PROPN':wn.NOUN,
    'VERB':wn.VERB}

def get_wordnet_token(token):
    return wn.synsets(token.lemma_,pos=wordnet_tags[token.pos_])

def get_word_net_similarity(token1, token2):
    word1 = get_wordnet_token(token1)
    word2 = get_wordnet_token(token2)
    return word1[0].wup_similarity(word2[0])

In [597]:
def compare_words(token1, token2):
    wordnet_sim_fail=False
    wcv_sim_fail=False
    
    if token1.lemma_ == token2.lemma_:
        return 1
    
    try:
        wcv_sim = model.wv.similarity(token1.lemma_, token2.lemma_)
    except:
        wcv_sim=None
        wcv_sim_fail=True
        
    try:
        wordnet_sim = get_word_net_similarity(token1,token2)
    except:
        wordnet_sim =None
        wordnet_sim_fail=True
        
    if (wcv_sim_fail or wcv_sim==None)  and (wordnet_sim==None or wordnet_sim_fail):
        return 0
    
    if wcv_sim_fail or wcv_sim==None:
        return wordnet_sim
    
    if wordnet_sim_fail or wordnet_sim == None:
         return wcv_sim
    
    return max(wcv_sim, wordnet_sim)

In [604]:
nlp_doc = get_doc('A man poses in front of an ad.')
roots = get_roots(nlp_doc)
parts1 =get_parts(nlp_doc,roots)
parts1

[{'obj_dep': [front, ad], 'subj_dep': [man], 'vb_dep': [poses]}]

In [605]:
parts1[0]['obj_dep'][1]

ad

In [606]:
nlp_doc = get_doc('An old man with a package poses in front of an advertisement.')
roots = get_roots(nlp_doc)
parts2 =get_parts(nlp_doc,roots)
parts2

[{'obj_dep': [front, advertisement], 'subj_dep': [man], 'vb_dep': [poses]}]

In [607]:
parts2[0]['obj_dep'][1]

advertisement

In [608]:
compare_words(parts1[0]['obj_dep'][1],parts2[0]['obj_dep'][1])

/Users/grycshuknazar/anaconda2/envs/env_36/lib/python3.6/site-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if __name__ == '__main__':


1.0

# Get block similarity

In [590]:
def get_block_similarity(block1, block2):
    if len(block1)==0 or len(block2)==0:
        return 0, None
    skiped_block2_indexes = []
    max_in_row=[]
    result_matrix =[]
    for id1,token_block1 in enumerate(block1):
        result_matrix.append([])
        for id2, token_block2 in enumerate(block2):
            if id2 in skiped_block2_indexes:
                result_matrix[id1].append(0)
            else:
                result_matrix[id1].append(compare_words(token_block1,token_block2))
                
        max_index = np.argmax(result_matrix[id1])


        skiped_block2_indexes.append(max_index)
        max_in_row.append(result_matrix[id1][max_index])
    return np.mean(max_in_row), result_matrix

In [547]:
res, matrix = get_block_similarity(parts1[0]['obj_dep'],parts2[0]['obj_dep'])
res

/Users/grycshuknazar/anaconda2/envs/env_36/lib/python3.6/site-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if __name__ == '__main__':


0.06872196507308677

In [560]:
blok_names = ['obj_dep','subj_dep', 'vb_dep']

def get_main_hipotes_similarity(parts1, parts2):
    skiped_block2_indexes = []
    max_in_row=[]
    result_matrix =[]
    for id1,part1 in enumerate(parts1):
        result_matrix.append([])
        blocks=[]
        for id2, part2 in enumerate(parts2):
            block={}
            if id2 in skiped_block2_indexes:
                result_matrix[id1].append(0)
            else:
                block={}
                for block_name in blok_names:
                    block[block_name] = get_block_similarity(part1[block_name],part2[block_name])[0]
                #print(list(block.values()))
                result_matrix[id1].append(np.mean(list(block.values())))
            blocks.append(block)
        max_index = np.argmax(result_matrix[id1])
        skiped_block2_indexes.append(max_index)
        max_in_row.append(blocks[max_index])
    return max_in_row, result_matrix

In [549]:
nlp_doc = get_doc('Three firefighters putting out a fire inside of a subway station')
roots = get_roots(nlp_doc)
parts2 =get_parts(nlp_doc,roots)
parts2

[{'obj_dep': [fire], 'subj_dep': [firefighters], 'vb_dep': [putting out]},
 {'obj_dep': [subway station], 'subj_dep': [firefighters], 'vb_dep': []}]

In [550]:
parts1

[{'obj_dep': [front, ad], 'subj_dep': [man], 'vb_dep': [poses]}]

In [553]:
parts2

[{'obj_dep': [fire], 'subj_dep': [firefighters], 'vb_dep': [putting out]},
 {'obj_dep': [subway station], 'subj_dep': [firefighters], 'vb_dep': []}]

In [564]:
get_main_hipotes_similarity(parts1,parts2)

/Users/grycshuknazar/anaconda2/envs/env_36/lib/python3.6/site-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if __name__ == '__main__':


([{'obj_dep': 0.06872196507308677,
   'subj_dep': 0.631578947368421,
   'vb_dep': 0.3333333333333333}],
 [[0.3445447485916137, 0.22969456238801023]])

In [571]:
def gry_similarity_feature(features, parts1,parts2):
    similarity=None
    if len(parts2)>len(parts1):
        similarity, scores = get_main_hipotes_similarity(parts1,parts2)
    else:
        similarity, scores = get_main_hipotes_similarity(parts2,parts1)
    for idx,item in enumerate(similarity):
        for part_type in item:
            features['similarity_{0}_{1}'.format(idx,part_type)]=item[part_type]
    for idr,row in enumerate(scores):
        for idx,item in enumerate(row):
            features['avg_scores_{0}_{1}'.format(idr,idx)]=item
    return features

In [570]:
gry_similarity_feature(parts1,parts2)

/Users/grycshuknazar/anaconda2/envs/env_36/lib/python3.6/site-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if __name__ == '__main__':


{'avg_scores_0_0': 0.3445447485916137,
 'avg_scores_0_1': 0.22969456238801023,
 'similarity_0_obj_dep': 0.06872196507308677,
 'similarity_0_subj_dep': 0.631578947368421,
 'similarity_0_vb_dep': 0.3333333333333333}

# get length feature

In [529]:
text = 'The statue is offensive and people are mad that it is on display'
nlp_doc = get_doc(text)
roots = get_roots(nlp_doc)
parts =get_parts(nlp_doc,roots)
parts

[{'obj_dep': [offensive], 'subj_dep': [statue], 'vb_dep': [is]},
 {'obj_dep': [mad], 'subj_dep': [people], 'vb_dep': [are]}]

In [573]:
def get_length_features(features, text, parts, prefix):

    text_len = len(text)
    features[prefix+'text_len']=text_len

    groups_count = len(parts)
    features[prefix+'groups_count']=groups_count

    for i in range(0,groups_count):
        for key in parts[i]:
            features[prefix+'{0}_{1}_len'.format(i,key)]=len(parts[i][key])
    
    return features

get_length_features({}, text,parts, 'main_')

{'main_0_obj_dep_len': 1,
 'main_0_subj_dep_len': 1,
 'main_0_vb_dep_len': 1,
 'main_1_obj_dep_len': 1,
 'main_1_subj_dep_len': 1,
 'main_1_vb_dep_len': 1,
 'main_groups_count': 2,
 'main_text_len': 64}

# Collect and vectorize features 

In [574]:
def get_features(text1,text2):
    features={}
    text1=str(text1)
    text2=str(text2)
    nlp_doc = get_doc(text1)
    roots = get_roots(nlp_doc)
    parts1 =get_parts(nlp_doc,roots)
    features = get_length_features(features, text1, parts1, 'main_')
    
    nlp_doc = get_doc(text2)
    roots = get_roots(nlp_doc)
    parts2 =get_parts(nlp_doc,roots)
    features = get_length_features(features, text2, parts2, 'hip_')
    
    features = gry_similarity_feature(features,parts1,parts2)
    return features

In [577]:
get_features('The statue is offensive and people are mad that it is on display','Three firefighters putting out a fire inside of a subway station')

/Users/grycshuknazar/anaconda2/envs/env_36/lib/python3.6/site-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if __name__ == '__main__':


{'avg_scores_0_0': 0.30868481724189223,
 'avg_scores_0_1': 0.22480022253793977,
 'avg_scores_1_0': 0,
 'avg_scores_1_1': 0.10711761372303215,
 'hip_0_obj_dep_len': 1,
 'hip_0_subj_dep_len': 1,
 'hip_0_vb_dep_len': 1,
 'hip_1_obj_dep_len': 1,
 'hip_1_subj_dep_len': 1,
 'hip_1_vb_dep_len': 0,
 'hip_groups_count': 2,
 'hip_text_len': 64,
 'main_0_obj_dep_len': 1,
 'main_0_subj_dep_len': 1,
 'main_0_vb_dep_len': 1,
 'main_1_obj_dep_len': 1,
 'main_1_subj_dep_len': 1,
 'main_1_vb_dep_len': 1,
 'main_groups_count': 2,
 'main_text_len': 64,
 'similarity_0_obj_dep': 0.1260544517256766,
 'similarity_0_subj_dep': 0.4,
 'similarity_0_vb_dep': 0.4,
 'similarity_1_obj_dep': 0.13953465935091464,
 'similarity_1_subj_dep': 0.18181818181818182,
 'similarity_1_vb_dep': 0}

In [581]:
from tqdm import tqdm

In [612]:
def get_X_y(arr):
    X=[]
    y=[]
    for idx in tqdm(range(0,len(arr))):
        try:
            item=arr[idx]
            X.append(get_features(item[0],item[1]))
            y.append(item[2])
        except:
            print(idx)
    return X,y

In [613]:
import gc
gc.collect()

413

In [614]:
X_train, y_train = get_X_y(train[['sentence1','sentence2','label1']].values)

  0%|          | 0/550152 [00:00<?, ?it/s]/Users/grycshuknazar/anaconda2/envs/env_36/lib/python3.6/site-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if __name__ == '__main__':
 17%|█▋        | 91490/550152 [25:10<2:06:10, 60.58it/s]

91479
91480
91481


 57%|█████▋    | 311133/550152 [1:26:50<1:06:42, 59.71it/s]

311124
311125
311126


100%|██████████| 550152/550152 [2:34:30<00:00, 59.34it/s]  


In [611]:
X_train

[{'avg_scores_0_0': 0.4344420298772225,
  'hip_0_obj_dep_len': 3,
  'hip_0_subj_dep_len': 1,
  'hip_0_vb_dep_len': 1,
  'hip_groups_count': 1,
  'hip_text_len': 49,
  'main_0_obj_dep_len': 2,
  'main_0_subj_dep_len': 1,
  'main_0_vb_dep_len': 1,
  'main_groups_count': 1,
  'main_text_len': 54,
  'similarity_0_obj_dep': 0.10332608963166771,
  'similarity_0_subj_dep': 1.0,
  'similarity_0_vb_dep': 0.2},
 {'avg_scores_0_0': 0.5246376811594202,
  'avg_scores_0_1': 0.4222222222222223,
  'hip_0_obj_dep_len': 1,
  'hip_0_subj_dep_len': 1,
  'hip_0_vb_dep_len': 1,
  'hip_1_obj_dep_len': 1,
  'hip_1_subj_dep_len': 1,
  'hip_1_vb_dep_len': 1,
  'hip_groups_count': 2,
  'hip_text_len': 45,
  'main_0_obj_dep_len': 2,
  'main_0_subj_dep_len': 1,
  'main_0_vb_dep_len': 1,
  'main_groups_count': 1,
  'main_text_len': 54,
  'similarity_0_obj_dep': 0.17391304347826086,
  'similarity_0_subj_dep': 1.0,
  'similarity_0_vb_dep': 0.4},
 {'avg_scores_0_0': 0.5654320987654321,
  'hip_0_obj_dep_len': 1,
  'hip

In [615]:
np.save('X_train.txt',X_train)

In [616]:
X_test, y_test = get_X_y(test[['sentence1','sentence2','label1']].values)

  0%|          | 0/10000 [00:00<?, ?it/s]/Users/grycshuknazar/anaconda2/envs/env_36/lib/python3.6/site-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if __name__ == '__main__':
100%|██████████| 10000/10000 [02:46<00:00, 60.10it/s]


In [619]:
from sklearn.feature_extraction import DictVectorizer
feature_vectorizer= DictVectorizer()
X_train_ = feature_vectorizer.fit_transform(X_train)

In [620]:
X_test_ = feature_vectorizer.transform(X_test)

In [623]:
from sklearn.preprocessing import LabelEncoder

In [630]:
label_encoder=LabelEncoder()
y_train_ = label_encoder.fit_transform(y_train)

In [631]:
y_test_ = label_encoder.transform(y_test)

In [637]:
from sklearn.linear_model import SGDClassifier

In [643]:
from sklearn.ensemble import RandomForestClassifier

In [691]:
lrm = RandomForestClassifier(max_depth=20, n_estimators=20, n_jobs=4)

In [692]:
lrm.fit(X_train_, y_train_)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=20, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=20, n_jobs=4,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [693]:
predicted = lrm.predict(X_train_)
from sklearn.metrics import classification_report
target_names = label_encoder.classes_
print(classification_report(y_train_, predicted, target_names=target_names))

               precision    recall  f1-score   support

contradiction       0.59      0.69      0.64    183382
   entailment       0.68      0.67      0.68    183382
      neutral       0.65      0.55      0.60    183382

  avg / total       0.64      0.64      0.64    550146



In [694]:
predicted = lrm.predict(X_test_)

In [695]:
from sklearn.metrics import classification_report
target_names = label_encoder.classes_
print(classification_report(y_test_, predicted, target_names=target_names))

               precision    recall  f1-score   support

contradiction       0.51      0.59      0.54      3333
   entailment       0.60      0.58      0.59      3333
      neutral       0.52      0.45      0.48      3334

  avg / total       0.54      0.54      0.54     10000



# LR

In [714]:
from sklearn.linear_model import LogisticRegression

In [715]:
lrm = LogisticRegression()

In [716]:
lrm.fit(X_train_, y_train_)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [717]:
predicted = lrm.predict(X_test_)
from sklearn.metrics import classification_report
target_names = label_encoder.classes_
print(classification_report(y_test_, predicted, target_names=target_names))

               precision    recall  f1-score   support

contradiction       0.46      0.50      0.48      3333
   entailment       0.52      0.59      0.55      3333
      neutral       0.51      0.40      0.45      3334

  avg / total       0.50      0.50      0.49     10000



In [ ]:
NN

In [696]:
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils

Using TensorFlow backend.


In [698]:
from keras.layers import Dropout
dummy_y = np_utils.to_categorical(y_train_)

In [705]:

#NN
# define baseline model
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(121, input_dim=121, activation='relu'))
    model.add(Dense(242, input_dim=242, activation='relu'))
    model.add(Dense(3, activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [708]:
estimator = KerasClassifier(build_fn=baseline_model, epochs=5, batch_size=10, verbose=1)

In [709]:
history = estimator.fit( X_train_, dummy_y)

Epoch 1/5
550146/550146 [==============================] - 103s 187us/step - loss: 1.0129 - acc: 0.4885
Epoch 2/5
373880/550146 [===================>..........] - ETA: 33s - loss: 0.9942 - acc: 0.5037

KeyboardInterrupt: 